In [2]:
import cv2
from keras.models import load_model
import numpy as np
from collections import deque

Using TensorFlow backend.


In [3]:
model1=load_model('devanagari.h5')
print(model1)

In [4]:
letter_count={0:'CHECK', 1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna', 6: 'character_06_cha',
                    7: '07_chha', 8: '08_ja', 9: '09_jha', 10: '10_yna',
                    11: '11_taamatar',
                    12: '12_thaa', 13: '13_daa', 14: '14_dhaa', 15: '15_adna', 16: '16_tabala', 17: '17_tha',
                    18: '18_da',
                    19: '19_dha', 20: '20_na', 21: '21_pa', 22: '22_pha',
                    23: '23_ba',
                    24: '24_bha', 25: '25_ma', 26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
                    31: '31_petchiryakha',32: '32_patalosaw', 33: '33_ha',
                    34: '34_chhya', 35: '35_tra', 36: '36_gya', 37:'CHECK'}

In [5]:
def keras_presict(model, image):
    processed=keras_process_image(image)
    print("processed:" + str(processed.shape))
    pred_probab=model.predict(processed)[0]
    pred_class=list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

def keras_process_image(img):
    image_x=32
    image_y=32
    img=cv2.resize(img, (image_x, image_y))
    img=np.array(img, dtype=np.float32)
    img=np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
cap=cv2.VideoCapture(0)
Lower_blue=np.array([110, 50, 50])
Upper_blue=np.array([130, 255, 255])
pred_class=0
pts=deque(maxlen=512)
blackboard=np.zeros((480, 640, 3), dtype=np.uint8)
digit=np.zeros((200, 200, 3), dtype=np.uint8)
while (cap.isOpened()):
    ret, img=cap.read()
    img=cv2.flip(img, 1)
    imgHSV=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(imgHSV, Lower_blue, Upper_blue)
    blur=cv2.medianBlur(mask, 15)
    blur=cv2.GaussianBlur(blur, (5, 5), 0)
    thresh=cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts=cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
    print(cnts)
    center=None
    if len(cnts) >= 1:
        contour = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(contour)>250:
            ((x, y), radius)=cv2.minEnclosingCircle(contour)
            cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
            M=cv2.moments(contour)
            center=(int(M['m10']/M['m00']), int(M['m01']/M['m00']))
            pts.appendleft(center)
            for i in range(1, len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard, pts[i-1], pts[i], (255, 255, 255), 10)
                cv2.line(img, pts[i-1], pts[i], (0, 0, 255), 5)
    elif len(cnts)==0:
        if len(pts)!=[]:
            blackboard_gray=cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1=cv2.medianBlur(blackboard_gray, 15)
            blur1=cv2.GaussianBlur(blur1, (5, 5), 0)
            thresh1=cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts=cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts)>=1:
                cnt=max(blackboard_cnts, key=cv2.contourArea)
                print(cv2.contourArea(cnt))
                if cv2.contourArea(cnt)>2000:
                    x, y, w, h=cv2.boundingRect(cnt)
                    digit=blackboard_gray[y:y +h, x:x +w]
                    pred_probab, pred_class=keras_predict(model1, digit)
                    print(pred_class, pred_probab)
        pts=deque(maxlen=512)
        blackboard=np.zeros((480, 640, 3), dtype=np.uint8)
    cv2.putText(img, "Conv Network:" + str(letter_count[pred_class]), (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k=cv2.waitKey(10)
    if k==27:
        break
            

[]
[]
[]
[]
[]
[]
[]
[]
[array([[[334, 166]],

       [[334, 167]],

       [[334, 168]],

       [[335, 169]],

       [[336, 170]],

       [[337, 170]],

       [[338, 170]],

       [[339, 170]],

       [[340, 169]],

       [[341, 168]],

       [[340, 167]],

       [[340, 166]],

       [[339, 166]],

       [[338, 166]],

       [[337, 166]],

       [[336, 166]],

       [[335, 166]]], dtype=int32), array([[[88,  0]],

       [[88,  1]],

       [[89,  1]],

       [[90,  1]],

       [[91,  1]],

       [[92,  0]],

       [[91,  0]],

       [[90,  0]],

       [[89,  0]]], dtype=int32)]
[]
[]
[]
[array([[[358,   0]],

       [[359,   1]],

       [[360,   1]],

       [[361,   0]],

       [[360,   0]],

       [[359,   0]]], dtype=int32)]
[]
[array([[[467,   0]],

       [[468,   1]],

       [[469,   1]],

       [[470,   0]],

       [[469,   0]],

       [[468,   0]]], dtype=int32)]
[]
[array([[[445,   0]],

       [[445,   1]],

       [[446,   2]],

       [[447,   3

[array([[[405,   0]],

       [[405,   1]],

       [[406,   1]],

       [[407,   2]],

       [[408,   2]],

       [[409,   2]],

       [[410,   1]],

       [[411,   1]],

       [[411,   0]],

       [[410,   0]],

       [[409,   0]],

       [[408,   0]],

       [[407,   0]],

       [[406,   0]]], dtype=int32)]
[array([[[359,   0]],

       [[359,   1]],

       [[360,   1]],

       [[361,   1]],

       [[362,   1]],

       [[362,   0]],

       [[361,   0]],

       [[360,   0]]], dtype=int32)]
[array([[[359,   0]],

       [[360,   1]],

       [[361,   1]],

       [[362,   1]],

       [[363,   1]],

       [[364,   2]],

       [[365,   2]],

       [[366,   1]],

       [[367,   1]],

       [[368,   0]],

       [[367,   0]],

       [[366,   0]],

       [[365,   0]],

       [[364,   0]],

       [[363,   0]],

       [[362,   0]],

       [[361,   0]],

       [[360,   0]]], dtype=int32)]
[array([[[359,   0]],

       [[360,   1]],

       [[361,   1]],

       [

[array([[[550, 345]],

       [[549, 346]],

       [[549, 347]],

       [[548, 348]],

       [[548, 349]],

       [[547, 350]],

       [[548, 351]],

       [[548, 352]],

       [[548, 353]],

       [[549, 354]],

       [[549, 355]],

       [[548, 356]],

       [[548, 357]],

       [[549, 358]],

       [[550, 359]],

       [[550, 360]],

       [[551, 361]],

       [[551, 362]],

       [[551, 363]],

       [[552, 363]],

       [[552, 362]],

       [[551, 361]],

       [[552, 360]],

       [[552, 359]],

       [[553, 358]],

       [[553, 357]],

       [[554, 356]],

       [[555, 355]],

       [[555, 354]],

       [[555, 353]],

       [[555, 352]],

       [[555, 351]],

       [[555, 350]],

       [[555, 349]],

       [[555, 348]],

       [[555, 347]],

       [[555, 346]],

       [[554, 345]],

       [[553, 345]],

       [[552, 345]],

       [[551, 345]]], dtype=int32)]
[array([[[550, 345]],

       [[549, 346]],

       [[549, 347]],

       [[548, 34

[array([[[529, 300]],

       [[528, 301]],

       [[527, 301]],

       [[526, 301]],

       [[525, 301]],

       [[524, 302]],

       [[523, 302]],

       [[522, 302]],

       [[521, 303]],

       [[520, 304]],

       [[519, 304]],

       [[518, 305]],

       [[517, 306]],

       [[516, 306]],

       [[515, 307]],

       [[514, 308]],

       [[513, 309]],

       [[512, 310]],

       [[511, 311]],

       [[510, 312]],

       [[509, 313]],

       [[508, 314]],

       [[508, 315]],

       [[507, 316]],

       [[507, 317]],

       [[507, 318]],

       [[506, 319]],

       [[506, 320]],

       [[506, 321]],

       [[507, 322]],

       [[507, 323]],

       [[507, 324]],

       [[508, 325]],

       [[509, 326]],

       [[510, 327]],

       [[511, 327]],

       [[512, 328]],

       [[513, 328]],

       [[514, 328]],

       [[515, 329]],

       [[516, 329]],

       [[517, 329]],

       [[518, 329]],

       [[519, 329]],

       [[520, 329]],

       [[

[array([[[530, 311]],

       [[529, 312]],

       [[528, 312]],

       [[527, 313]],

       [[526, 313]],

       [[525, 314]],

       [[524, 315]],

       [[523, 315]],

       [[522, 316]],

       [[521, 317]],

       [[520, 318]],

       [[519, 319]],

       [[519, 320]],

       [[518, 321]],

       [[517, 322]],

       [[516, 323]],

       [[516, 324]],

       [[515, 325]],

       [[514, 326]],

       [[514, 327]],

       [[513, 328]],

       [[513, 329]],

       [[513, 330]],

       [[513, 331]],

       [[513, 332]],

       [[513, 333]],

       [[513, 334]],

       [[513, 335]],

       [[513, 336]],

       [[513, 337]],

       [[514, 338]],

       [[514, 339]],

       [[515, 340]],

       [[516, 341]],

       [[517, 341]],

       [[518, 342]],

       [[519, 343]],

       [[520, 343]],

       [[521, 343]],

       [[522, 343]],

       [[523, 343]],

       [[524, 343]],

       [[525, 343]],

       [[526, 343]],

       [[527, 343]],

       [[

...

[array([[[606, 446]],

       [[605, 447]],

       [[604, 447]],

       [[603, 447]],

       [[602, 447]],

       [[601, 447]],

       [[600, 448]],

       [[599, 448]],

       [[598, 449]],

       [[597, 449]],

       [[596, 450]],

       [[595, 451]],

       [[594, 452]],

       [[594, 453]],

       [[594, 454]],

       [[593, 455]],

       [[593, 456]],

       [[594, 457]],

       [[595, 458]],

       [[595, 459]],

       [[596, 460]],

       [[597, 461]],

       [[598, 462]],

       [[599, 463]],

       [[600, 464]],

       [[601, 465]],

       [[602, 466]],

       [[603, 466]],

       [[604, 467]],

       [[605, 468]],

       [[606, 468]],

       [[607, 469]],

       [[608, 469]],

       [[609, 470]],

       [[610, 470]],

       [[611, 470]],

       [[612, 470]],

       [[613, 471]],

       [[614, 471]],

       [[615, 471]],

       [[616, 471]],

       [[617, 472]],

       [[618, 472]],

       [[619, 473]],

       [[620, 474]],

       [[

[array([[[598, 431]],

       [[597, 432]],

       [[596, 432]],

       [[595, 432]],

       [[594, 432]],

       [[593, 432]],

       [[592, 433]],

       [[591, 433]],

       [[590, 433]],

       [[589, 434]],

       [[588, 434]],

       [[587, 435]],

       [[586, 435]],

       [[585, 436]],

       [[584, 437]],

       [[583, 438]],

       [[583, 439]],

       [[583, 440]],

       [[583, 441]],

       [[584, 442]],

       [[584, 443]],

       [[585, 443]],

       [[586, 444]],

       [[587, 445]],

       [[588, 446]],

       [[589, 447]],

       [[590, 447]],

       [[591, 448]],

       [[592, 448]],

       [[593, 449]],

       [[594, 449]],

       [[595, 450]],

       [[596, 450]],

       [[597, 451]],

       [[598, 451]],

       [[599, 452]],

       [[600, 452]],

       [[601, 452]],

       [[602, 453]],

       [[603, 453]],

       [[604, 453]],

       [[605, 454]],

       [[606, 454]],

       [[607, 454]],

       [[608, 455]],

       [[

[array([[[582, 462]],

       [[581, 463]],

       [[580, 463]],

       [[579, 463]],

       [[578, 463]],

       [[577, 464]],

       [[576, 464]],

       [[575, 464]],

       [[574, 465]],

       [[573, 465]],

       [[572, 466]],

       [[572, 467]],

       [[571, 468]],

       [[571, 469]],

       [[571, 470]],

       [[571, 471]],

       [[571, 472]],

       [[572, 473]],

       [[572, 474]],

       [[573, 475]],

       [[574, 476]],

       [[575, 477]],

       [[576, 478]],

       [[576, 479]],

       [[577, 479]],

       [[578, 479]],

       [[579, 479]],

       [[580, 479]],

       [[581, 479]],

       [[582, 479]],

       [[583, 479]],

       [[584, 479]],

       [[585, 479]],

       [[586, 479]],

       [[587, 479]],

       [[588, 479]],

       [[589, 479]],

       [[590, 479]],

       [[591, 479]],

       [[592, 479]],

       [[593, 479]],

       [[594, 479]],

       [[595, 479]],

       [[596, 479]],

       [[597, 479]],

       [[

[array([[[587, 467]],

       [[586, 468]],

       [[585, 468]],

       [[584, 468]],

       [[583, 468]],

       [[582, 468]],

       [[581, 468]],

       [[580, 469]],

       [[579, 469]],

       [[578, 469]],

       [[577, 469]],

       [[576, 470]],

       [[575, 470]],

       [[574, 470]],

       [[573, 471]],

       [[572, 472]],

       [[571, 472]],

       [[570, 473]],

       [[569, 474]],

       [[568, 475]],

       [[567, 476]],

       [[567, 477]],

       [[566, 478]],

       [[566, 479]],

       [[567, 479]],

       [[568, 479]],

       [[569, 479]],

       [[570, 479]],

       [[571, 479]],

       [[572, 479]],

       [[573, 479]],

       [[574, 479]],

       [[575, 479]],

       [[576, 479]],

       [[577, 479]],

       [[578, 479]],

       [[579, 479]],

       [[580, 479]],

       [[581, 479]],

       [[582, 479]],

       [[583, 479]],

       [[584, 479]],

       [[585, 479]],

       [[586, 479]],

       [[587, 479]],

       [[

NameError: name 'keras_predict' is not defined